<a href="https://colab.research.google.com/github/BruceXavierChou/stock_analysis/blob/main/%E3%80%8Cstk_ch05_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-05 AI 技術指標回測系統

## 5-2 強大的回測工具：backtesting.py

### 1️⃣ 安裝及匯入套件

In [1]:
!pip install openai
!pip install yfinance
!pip install backtesting
!pip install bokeh==2.4.3 # 繪圖套件
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import datetime as dt # 時間套件
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=3621cd70706eda1b95366e64bd24b6508427a6a1913d28a7c8b244c86bbf8047
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.4.3
    Uninstalling bokeh-3.4.3:
      Successfully uninstalled bokeh-3.4.3
ERROR: pip's dependency resolver does

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### 2️⃣ 取得股價資料

In [2]:
# 輸入股票代號
# stock_id = "2330.tw"
stock_id = "NVDA"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
df['ma1'] = df['Close'].rolling(window=5).mean()
df['ma2'] = df['Close'].rolling(window=10).mean()
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ma1,ma2
Date,,,,,,,,
2019-08-06,3.84500,3.89075,3.76075,3.80875,3.786890,361272000,NaN,NaN
2019-08-07,3.74750,3.85925,3.73000,3.84725,3.825169,372560000,NaN,NaN
2019-08-08,3.90525,3.95775,3.84750,3.95650,3.933792,309300000,NaN,NaN
2019-08-09,3.91525,3.91875,3.80375,3.85450,3.832377,341816000,NaN,NaN
2019-08-12,3.80200,3.85450,3.76425,3.78625,3.764519,279200000,3.85065,NaN


### 3️⃣ 定義回測策略

In [3]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
      self.buy(size=1)
    elif crossover(self.data.ma2, self.data.ma1):
      self.sell(size=1)

### 4️⃣ 回測結果

In [4]:
backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   41.732909
Equity Final [$]                100001.481627
Equity Peak [$]                 100002.489898
Return [%]                           0.001482
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                    0.000297
Volatility (Ann.) [%]                0.014763
Sharpe Ratio                         0.020104
Sortino Ratio                        0.031028
Calmar Ratio                         0.009136
Max. Drawdown [%]                   -0.032487
Avg. Drawdown [%]                   -0.011758
Max. Drawdown Duration      961 days 00:00:00
Avg. Drawdown Duration      449 days 00:00:00
# Trades                                   67
Win Rate [%]                        28.358209
Best Trade [%]                      24.144672
Worst Trade [%]                    -18.952708
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,18,22,4.204614,4.47000,-0.265386,-0.063118,2019-08-30,2019-09-06,7 days
1,-1,32,43,4.404810,4.61450,-0.209690,-0.047605,2019-09-20,2019-10-07,17 days
2,-1,73,74,5.076861,5.27575,-0.198889,-0.039176,2019-11-18,2019-11-19,1 days
3,-1,85,90,5.236470,5.40450,-0.168030,-0.032088,2019-12-05,2019-12-12,7 days
4,-1,105,107,5.784768,5.99400,-0.209232,-0.036169,2020-01-06,2020-01-08,2 days


### 5️⃣ 回測繪圖

In [5]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

Row(id='1407', ...)

### 6️⃣ 設定停利、停損點

In [6]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
        # 買入時設置停損與停利價格
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.ma2, self.data.ma1):
        # 賣出時時設置停損與停利價格
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)

backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()
print(stats)

Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   43.163752
Equity Final [$]                100046.085032
Equity Peak [$]                 100046.085032
Return [%]                           0.046085
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                     0.00923
Volatility (Ann.) [%]                0.011536
Sharpe Ratio                         0.800092
Sortino Ratio                        1.349241
Calmar Ratio                         0.801641
Max. Drawdown [%]                   -0.011514
Avg. Drawdown [%]                   -0.002693
Max. Drawdown Duration      724 days 00:00:00
Avg. Drawdown Duration       67 days 00:00:00
# Trades                                   87
Win Rate [%]                        42.528736
Best Trade [%]                      16.485003
Worst Trade [%]                    -13.153138
Avg. Trade [%]                    

## 5-3 讓 AI 產生回測策略

### 7️⃣ 輸入 OpenAI API KEY

In [7]:
import getpass
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key=api_key)

請輸入金鑰：··········


### 8️⃣ 創建 GPT 3.5 模型函式

In [8]:
# GPT 3.5 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                         messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def ai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
      Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}.\
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content."
  }]

  reply_data = get_reply(msg)
  return reply_data

# 產生技術指標策略
def ai_strategy(df, user_msg, add_msg="無"):

  code_example ='''
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.short_ma, self.data.long_ma):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.long_ma, self.data.short_ma):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)
        '''

  msg = [{
    "role":
    "system",
    "content":
     "As a Python code generation bot, your task is to generate \
     code for a stock strategy based on user requirements and df. \
     Please note that your response should solely \
     consist of the code itself, \
     and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
     "The user requirement:計算 ma,\n\
     The additional requirement: 請設置 10% 的停利與停損點\n\
     The df.columns =['Open',	'High', 'Low',	'Close',	'Adj Close',	'Volume', 'short_ma',	'long_ma']\n\
     Please using the crossover() function in next(self)\
     Your response should strictly contain the Python \
     code for the 'AiStrategy(Strategy)' class \
     and exclude any unrelated content."
  }, {
    "role":
    "assistant",
    "content":f"{code_example}"
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg}\n\
     The additional requirement:{add_msg}\n\
     The df.columns ={df.columns}\n\
     Your task is to develop a Python class named \
     'AiStrategy(Strategy)'\
     Please using the crossover() function in next(self)."

  }]

  reply_data = get_reply(msg)
  return reply_data


### 9️⃣ 計算技術指標

In [9]:
# 輸入股票代號
# stock_id = "2330.tw"
stock_id = "NVDA"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
user_msg = ["MACD", "請設置10%的停損點與20%的停利點"]
code_str = ai_helper(df, user_msg[0])
print(code_str)
exec(code_str)
new_df = calculate(df)
new_df.tail()

[*********************100%%**********************]  1 of 1 completed


def calculate(df):
    short_ema = df['Close'].ewm(span=12, adjust=False).mean()
    long_ema = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = short_ema - long_ema
    return df


,Open,High,Low,Close,Adj Close,Volume,MACD
Date,,,,,,,
2024-07-30,111.519997,111.989998,102.540001,103.730003,103.730003,486833300,-2.954132
2024-07-31,112.900002,118.339996,110.879997,117.019997,117.019997,473174200,-2.667543
2024-08-01,117.529999,120.160004,106.809998,109.209999,109.209999,523462300,-3.035627
2024-08-02,103.760002,108.720001,101.370003,107.269997,107.269997,482027500,-3.444176
2024-08-05,92.059998,103.410004,90.690002,100.449997,100.449997,549739200,-4.269060


### 🔟 策略生成

In [10]:
strategy_str = ai_strategy(new_df, user_msg[0], user_msg[1])
print(strategy_str)
print("-----------------------")
exec(strategy_str)
backtest = Backtest(df,
        AiStrategy,
        cash=100000,
        commission=0.004,
        trade_on_close=True,
        exclusive_orders=True,
        )
stats = backtest.run()
print(stats)

class AiStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(self.data.MACD, 0):
            self.buy(size=1,
                     sl=self.data.Close[-1] * 0.90,
                     tp=self.data.Close[-1] * 1.20)
        elif crossover(0, self.data.MACD):
            self.sell(size=1,
                      sl=self.data.Close[-1] * 1.10,
                      tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   40.143084
Equity Final [$]                  100028.9755
Equity Peak [$]                   100028.9755
Return [%]                           0.028975
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                    0.005804
Volatility (Ann.) [%]                 0.01369
Sharpe Ratio                         0.423935
Sortino Ratio            

### 1️⃣1️⃣ 寫成函式

In [11]:
def ai_backtest(stock_id, period, user_msg, add_msg):

  # 下載資料
  df = yf.download(stock_id, period=period)

  # 獲取和執行指標計算程式碼
  code_str = ai_helper(df, user_msg)
  local_namespace = {}
  exec(code_str, globals(), local_namespace)
  calculate = local_namespace['calculate']
  new_df = calculate(df)

  # 獲取和執行策略程式碼
  strategy_str = ai_strategy(new_df, user_msg, add_msg)
  print(strategy_str)
  print("-----------------------")
  exec(strategy_str, globals(), local_namespace)
  AiStrategy = local_namespace['AiStrategy']

  backtest = Backtest(df,
          AiStrategy,
          cash=100000,
          commission=0.004,
          trade_on_close=True,
          exclusive_orders=True,
          )
  stats = backtest.run()
  print(stats)
  return str(stats)


## 5-4 讓 AI 解析回測報告

### 1️⃣2️⃣ 設定 AI 回復內容

In [12]:
def backtest_analysis(*args):

  content_list = [f"策略{i+1}：{report}"
                  for i, report in enumerate(args)]
  content = "\n".join(content_list)
  content += "\n\n請依資料給我一份約200字的分析報告。若有多個策略, \
                  請選出最好的策略及原因, reply in 繁體中文."

  msg = [{
      "role": "system",
      "content": "你是一位專業的證券分析師, 我會給你交易策略的回測績效,\
                  請幫我進行績效分析.不用詳細講解每個欄位, \
                  重點說明即可, 並回答交易策略的好壞"
  }, {
      "role": "user",
      "content": content
  }]

  reply_data = get_reply(msg)
  return reply_data


### 1️⃣3️⃣ 回測結果分析

In [13]:
#"2330.tw"可替換為其他股票代碼，包含美股，這邊我改分析NVIDIA(代碼:NVDA)

stats = ai_backtest(stock_id="NVDA",
           period="5y",
           user_msg="MACD",
           add_msg="請設置10%的停損點與20%的停利點")
reply = backtest_analysis(stats)
print(reply)


[*********************100%%**********************]  1 of 1 completed


class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.EMA_12, self.data.EMA_26):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.EMA_26, self.data.EMA_12):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   40.143084
Equity Final [$]                  100028.9755
Equity Peak [$]                   100028.9755
Return [%]                           0.028975
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                    0.005804
Volatility (Ann.) [%]                 0.01369
Sharpe Ratio                         0.423935
Sortino Ratio                        0.726334
Calma

### 1️⃣4️⃣ 比較多個策略

In [15]:
# 策略1:MACD+停利停損
stats1 = ai_backtest(stock_id="NVDA", period="5y",
            user_msg="MACD",
            add_msg="請設置10%的停損點與20%的停利點")
# 策略2:SMA
stats2 = ai_backtest(stock_id="NVDA", period="5y",
            user_msg="SMA",
            add_msg="無")
# 策略3:RSI+停利停損
stats3 = ai_backtest(stock_id="NVDA", period="5y",
            user_msg="RSI",
            add_msg="請設置10%的停損點與20%的停利點")

reply = backtest_analysis(stats1, stats2, stats3)
print(reply)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, 0):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(0, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)  
-----------------------
Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   40.143084
Equity Final [$]                  100028.9755
Equity Peak [$]                   100028.9755
Return [%]                           0.028975
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                    0.005804
Volatility (Ann.) [%]                 0.01369
Sharpe Ratio                         0.423935
Sortino Ratio                        0.726334
Calmar Ratio                         


[*********************100%%**********************]  1 of 1 completed

class AiStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(self.data.Close, self.data.SMA):
            self.buy()
        elif crossover(self.data.SMA, self.data.Close):
            self.sell()
-----------------------
Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   97.376789
Equity Final [$]                 325574.30284
Equity Peak [$]                 397151.433942
Return [%]                         225.574303
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                   26.675617
Volatility (Ann.) [%]               66.625508
Sharpe Ratio                         0.400381
Sortino Ratio                        0.822241
Calmar Ratio                         0.387891
Max. Drawdown [%]                   -68.77087
Avg. Drawdown [%]                  -11.390766
Max. Drawdown Duration      846 days 

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if self.data.RSI[-1] < 30:
        self.buy(size=1,
                 sl=self.data.Close[-1] * 0.90,
                 tp=self.data.Close[-1] * 1.20)
    elif self.data.RSI[-1] > 70:
        self.sell(size=1,
                  sl=self.data.Close[-1] * 1.10,
                  tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-08-06 00:00:00
End                       2024-08-05 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   91.653418
Equity Final [$]                 99901.018621
Equity Peak [$]                 100000.096785
Return [%]                          -0.098981
Buy & Hold Return [%]             2537.348191
Return (Ann.) [%]                   -0.019836
Volatility (Ann.) [%]                0.022083
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                   